In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 3-1. 모델 아키텍처 설계
- 스마트 야드 또는 기타 산업 환경에서 공장 부품을 자동으로 분류하고 식별하는 딥러닝 모델을 개발하는 경우, 실제 환경에서의 요구 사항과 정확도, 처리 속도, 신뢰성을 고려해야 합니다.
- 이러한 환경에 적합한 모델을 설계하기 위해 EfficientNet, ResNet, MobileNet 등과 같은 보다 정교한 아키텍처를 고려할 수 있습니다.
    - EfficientNet
      - 높은 정확도와 효율성을 제공합니다.
      - 다양한 크기의 옵션을 제공하므로, 프로젝트의 자원 및 정확도 요구 사항에 따라 적절한 모델을 선택할 수 있습니다.
    - ResNet
      - 깊은 신경망을 사용하는 복잡한 이미지 분류 작업에서 검증된 성능을 보여줍니다.
      - 특히 부품의 세부적인 특징을 학습하는 데 유리할 수 있습니다.
    - MobileNet
      - 매우 효율적이며 빠르게 동작합니다.
      - 특히 자원이 제한된 환경에서 유용합니다.
      - 다만, ResNet이나 EfficientNet에 비해 정확도가 조금 낮을 수 있습니다.

#### 3-1-1. EfficientNet을 이용한 모델 설계 및 저장
- 특징
  - 기존 모델의 스케일링 방법을 개선하여 균형 잡힌 방식으로 네트워크의 깊이, 너비, 해상도를 동시에 조정합니다.

- 모델 설계 주의사항
    - 스케일링 조정
    > EfficientNet은 기본적으로 복잡도를 조절하는 compound scaling 방법을 사용합니다.

    > 모델의 규모를 변경할 때는 네트워크의 너비, 깊이, 해상도를 동시에 조정해야하는데 이는 EfficientNetB0에서 EfficientNetB7까지 다양한 크기의 모델을 적절히 선택하여 사용하는 것을 의미합니다.
    - 하이퍼파라미터 튜
    > EfficientNet의 성능은 하이퍼파라미터에 민감할 수 있습니다.

    > 학습률, 배치 크기, 에포크 수 등의 파라미터를 실험적으로 조정하여 최적의 학습 결과를 도출해야 합니다.
    - 데이터 전처리
    > EfficientNet은 입력 이미지 크기에 따라 다양하게 설계되었습니다.

    > 각 모델 버전에 맞는 입력 크기를 사용해야 하며, 입력 데이터는 적절히 정규화되어야 합니다.

- 모델 컴파일 : 딥러닝에서 모델을 학습할 준비를 마치기 위해 필요한 설정을 모델에 적용하는 과정을 의미합니다.
    - 옵티마이저(Optimizer) > 모델의 파라미터를 업데이트하는 방식을 결정합니다.
    > 옵티마이저는 학습률과 같은 매개변수를 조정하여 학습 과정에서 손실 함수를 최소화하는 방향으로 모델의 가중치를 조정합니다.
    
    > 대표적인 옵티마이저로는 SGD(Stochastic Gradient Descent), Adam, RMSprop 등이 있습니다.
    - 손실 함수(Loss Function) > 모델의 예측 값과 실제 값 사이의 차이를 측정하는 함수로, 모델의 성능을 평가하는 기준입니다.
    > 손실 함수의 값이 낮을수록 모델의 성능이 좋다고 평가됩니다.
    
    > 회귀 문제에서는 주로 MSE(Mean Squared Error)나 MAE(Mean Absolute Error)를 사용하고, 분류 문제에서는 크로스 엔트로피(Cross-Entropy) 손실 함수를 사용합니다.
    - 평가 지표(Metrics) > 훈련 및 검증 과정을 모니터링하기 위해 사용되는 지표입니다.
    > 정확도(accuracy), 정밀도(precision), 재현율(recall), F1 점수 등이 일반적으로 사용됩니다.
- 모델 컴파일의 필요성과 타이밍
    - 새 모델 초기화 시
    > 이는 모델에 훈련 과정에서 사용할 옵티마이저, 손실 함수, 평가 지표를 설정하기 위함입니다.
    - 모델 아키텍처 변경 후
    > 이는 변경된 구조에 맞게 새로운 학습 설정을 적용하기 위함입니다.
    - 학습 파라미터 변경 시
    > 변경된 파라미터를 적용하여 모델을 최적화하기 위함입니다.

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall

def create_efficientnet_model(num_classes):
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = True  # 미세 조정을 위해 전체 모델을 훈련 가능하도록 설정

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
    return model

# 모델 생성 및 요약
model_EfficientNet = create_efficientnet_model(num_classes=4)
model_EfficientNet.summary()

16705208/16705208 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     

- TensorFlow/Keras에서는 정수 라벨 형태를 사용할 때, 기본적으로 'sparse_categorical_crossentropy' 손실함수를 사용합니다.
- But, Precision과 Recall에 대해서는 별도의 "sparse" 버전이 제공되지 않는다.

- 원-핫 인코딩 형식일 경우 'categorical_crossentropy' 손실함수를 사용합니다.
- 이 때는 Precision, Recall 그리고 CategoricalAccuracy 메트릭을 함께 사용할 수 있습니다.

In [ ]:
# 모델 아키텍처를 JSON 파일로 저장
model_json = model_EfficientNet.to_json()
with open("/content/drive/My Drive/project_classification-DL/models/model_EfficientNet_architecture.json", "w") as json_file:
    json_file.write(model_json)

# 모델의 가중치를 HDF5 파일로 저장
model_EfficientNet.save_weights('/content/drive/My Drive/project_classification-DL/models/model_EfficientNet.weights.h5')

#### 3-1-2. ResNet을 이용한 모델 설계 및 저장
- 특징
  - ResNet은 깊은 신경망에서 발생하는 소실된 기울기 문제를 해결하기 위해 "잔차 블록"(residual blocks)을 도입합니다.
  > 이 블록들은 입력을 출력에 직접 추가함으로써 네트워크가 학습을 더 잘하도록 돕습니다.
- 모델 설계 주의사항
    - 잔차 연결 구현
    > ResNet의 핵심은 잔차 연결입니다.

    >  이 연결은 신경망의 입력을 몇 층을 건너뛰어 출력에 직접 더해주는 방식으로 작동합니다.
    
    > 이러한 구조를 정확히 구현하여 깊은 네트워크에서도 정보가 효과적으로 전파될 수 있도록 해야 합니다.
    - 과적합 방지
    > 깊은 모델인 ResNet은 과적합의 위험이 있습니다.
    
    > 드롭아웃, 데이터 증강, 가중치 감쇠 등을 적절히 사용하여 모델의 일반화 능력을 향상시키는 것이 중요합니다.
    - 콜백과 학습률 조정
    >  훈련 중 학습률을 조절하거나 모델의 성능 향상을 모니터링하기 위해 콜백을 설정할 수 있습니다.
    
    > 이는 효율적인 모델 훈련을 위해 중요합니다.
    - 층수 선택
    > ResNet에는 여러 버전(예: ResNet-50, ResNet-101, ResNet-152 등)이 있고, 사용할 모델의 깊이를 데이터와 작업의 복잡성에 맞게 선택해야 합니다.

    > 너무 깊은 모델은 계산 비용이 높고, 학습이 어려울 수 있습니다.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall

def create_resnet_model(num_classes):
    # ResNet50 모델 로드, ImageNet 가중치 포함, 상위층 제외
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # 기반 모델의 파라미터를 동결 (학습에서 제외)
    for layer in base_model.layers:
        layer.trainable = False

    # 새로운 분류 층을 추가
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # 전체 모델을 생성
    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
    return model

# 모델 생성 및 요약
num_classes = 4  # 실제 프로젝트의 클래스 수에 따라 조정
model_ResNet = create_resnet_model(num_classes)
model_ResNet.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [ ]:
# 모델 아키텍처를 JSON 파일로 저장
model_json = model_ResNet.to_json()
with open("/content/drive/My Drive/project_classification-DL/models/model_ResNet_architecture.json", "w") as json_file:
    json_file.write(model_json)

# 모델의 가중치를 HDF5 파일로 저장
model_ResNet.save_weights('/content/drive/My Drive/project_classification-DL/models/model_ResNet.weights.h5')

#### 3-1-3. MobileNet을 이용한 모델 설계 및 저장
- 특징
  - MobileNet은 경량화된 네트워크로, 주로 모바일과 임베디드 디바이스를 위해 설계되었습니다.
  - Depthwise separable convolution을 사용하여 계산량과 모델 크기를 대폭 줄입니다.

- 모델 설계 주의사항
    - Depthwise Separable Convolution
    > MobileNet의 핵심은 Depthwise Separable Convolution입니다.

    > 이 기술은 표준 컨볼루션을 depthwise 컨볼루션과 pointwise 컨볼루션으로 분리하여 계산 효율성을 크게 향상시킵니다.
    - 모델 크기와 성능 조절
    > MobileNet은 다양한 크기와 정확도 설정을 제공합니다.
    - 하이퍼파라미터 튜닝
    > MobileNet은 비교적 작은 모델이지만, 옵티마이저 선택과 학습률 설정이 성능에 큰 영향을 미칠 수 있습니다.

    > Adam, SGD 등 다양한 옵티마이저를 실험해 보고, 적절한 학습률 스케줄을 적용하는 것이 중요합니다.

    > 또한, 오버피팅을 방지하기 위해 드롭아웃, 데이터 증강 등의 기법을 적절히 사용해야 합니다.

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall

def create_mobilenet_model(num_classes):
    # MobileNetV2를 기반 모델로 로드
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # 상위 층의 일부를 훈련 가능하도록 설정
    base_model.trainable = True
    fine_tune_at = 100  # 상위 100개 층을 훈련
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # 새로운 분류 층을 추가
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)  # 뉴런 수 조정
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # 전체 모델 구성
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
    return model

# 모델 생성 및 요약
model_MobileNet = create_mobilenet_model(num_classes=4)
model_MobileNet.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (N

In [ ]:
# 모델 아키텍처를 JSON 파일로 저장
model_json = model_MobileNet.to_json()
with open("/content/drive/My Drive/project_classification-DL/models/model_MobileNet_architecture.json", "w") as json_file:
    json_file.write(model_json)

# 모델의 가중치를 HDF5 파일로 저장
model_MobileNet.save_weights('/content/drive/My Drive/project_classification-DL/models/model_MobileNet.weights.h5')